### This Notebook is a mix of codes used at various places. It was an Application development company, they wanted to track all their Applications via different tracking platforms (Applovin, Ironsource, Unity Ads). So this notebook has API data pulls from all these platforms and storing data into their BigQuery datawarehouse.

*Note that all API keys/secret keys/auth token are expired and not exposed in this notebook.

In [ ]:
'abcd'.translate('a'.maketrans('abc','bcd'))

'bcdd'

In [ ]:
## This script will extract data from Applovin Mediation Platform
## and Iron Source Mediation Platform
## using their APIs and will stream data after processing to AppSwim
## BigQuery Datawarehouse. 

import json
import requests
import pandas as pd
import datetime
import base64
from google.cloud import bigquery
import pandas_gbq
from google import oauth2
import copy
import time
import re
import csv

In [ ]:
import random

In [ ]:
API_KEY = '---'
adver_url = "https://rest.crossinstall.io/v1/advertisers"
headers={"Api-Key": API_KEY}

resp = requests.get(adver_url, headers=headers)
adver_id = json.loads(resp.text)
adver_id

{'__actions__': {},
 '__class__': 'advertiser',
 '__href__': '/v1/advertisers/Bzkk',
 '__links__': {'advertiser_campaigns': '/v1/advertisers/Bzkk/campaigns'},
 'advertiser_id': 'Bzkk',
 'name': 'APP SWIM'}

In [ ]:
headers={"Api-Key": 'd9cd30e91a08984e30e201d2651504b5'}
camp_id = 'ADS'
stats_url = 'https://rest.crossinstall.io/v1/advertisers/Bzkk/campaigns/RSZk/stats/RSZk/1?page_size=10'
print(stats_url)
resp = requests.get(stats_url, headers=headers)
resp.text

https://rest.crossinstall.io/v1/advertisers/Bzkk/campaigns/RSZk/stats/RSZk/1?page_size=10


'{"__class__":"error","__href__":"","__links__":{},"__action__":{},"type":"http_error","code":"403","message":"Access Denied.","context":[]}'

In [ ]:

statsId = statsId_example # String | returned in `stats_id` field for a single campaign from
`POST /advertisers/{advertiser_id}/campaigns/{campaign_id}` API endpoint

page = page_example # String | page number for iterating through results

pageSize = pageSize_example # String | number of entries to return per page (50-400) (optional)

try:
    # Get a page of stats previously requested in postSingleAdvertiserCampaignStats endpoint
    api_response = api_instance.get_single_advertiser_campaign_stats_page(advertiserId, campaignId, statsId, page, pageSize=pageSize)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling StatsApi->getSingleAdvertiserCampaignStatsPage: %s\n" % e)

In [ ]:
## this file app-swim-bi-bq-key.json can be obtained from BigQuery
## console in service-account section.
## this code initialized BigQuery API client
bq_client = bigquery.Client.from_service_account_json('app-swim-bi-bq-key.json')

## CrossInstall Data Pull

In [ ]:
from __future__ import print_statement
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

# Configure API key authorization: ApiKeyAuth
api_key = '---'


## Ad Colony Data Pull

In [ ]:
url = 'https://clients-api.adcolony.com/api/v2/advertiser_summary?user_credentials=wuyxAyJbhAy7c8RBvzf8&dat\
e=07312020&end_date=08122020&format=json&group_by=campaign,ad_group,creative,country,platform&date_group=day'
resp = requests.get(url)

In [ ]:
adcolony_data = json.loads(resp.text)
adcolony_data

In [ ]:
adcolony_data['results'][0]

In [ ]:
job = bq_client.load_table_from_json(adcolony_data['results'], destination='app-swim-bi.adcolony_dataset.adcolony_data')
job.result();

## Applovin Max Data pull

In [ ]:
## we first get the latest date present in the Applovin data already
## we will use this date as startdate and now (means today) as enddate
## as parameters to Applovin Reporting API. This saves us from querying
## data from begining from API everytime.

## SQL query to get MAX date in the data
previous_max_date = 'SELECT MAX(day) FROM `app-swim-bi.dashboard_page9.applovinMAX_revenue_report` '
## getting query results in pandas dataframe
previous_max_date = bq_client.query(previous_max_date).to_dataframe()
## converting datetime format to string, by specifying format which is same as 
## present in the bigquery table
previous_max_date = datetime.date.strftime(previous_max_date.values[0][0], format='%Y-%m-%d')

In [ ]:
## SDK key for Applovin
sdk_key = '---'

## this is the Applovin Reporting API key
report_key = '---'

In [ ]:
today = datetime.date.strftime(datetime.date.today(),format='%Y-%m-%d')
prev_two_days = datetime.date.strftime(datetime.datetime.strptime(previous_max_date,'%Y-%m-%d') - datetime.timedelta(days=2),format='%Y-%m-%d')

In [ ]:
## the url provided by Applovin API to pull data from in json output
url = 'http://r.applovin.com/maxReport?api_key='+report_key+'&start='+prev_two_days+'&end='+today+\
      '&columns=day,application,network,impressions,estimated_revenue,ecpm,package_name,platform,network_placement,max_placement,'+\
      'max_ad_unit,max_ad_unit_id,max_ad_unit_test,device_type,ad_format,country'+\
      '&format=json'


## following columns are not included in the results, which could be 
## extracted but they are not necessary.
## columns omitted: hour,requests,attempts,responses,fill_rate
resp = requests.get(url)
report_data = json.loads(resp.text)

In [ ]:
## this code first query the table and selects only
## unique rows. Then write the results back to the table
## this way we removed duplicates in the data easily
#job_config = bigquery.QueryJobConfig(destination='app-swim-bi.dashboard_page9.applovinMAX_revenue_report', write_disposition='write_truncate')

de_duplication = "Delete `app-swim-bi.dashboard_page9.applovinMAX_revenue_report`\
                    Where day >= '"+str(prev_two_days)+ "'  AND day <= '" +str(today)+"'"
q = bq_client.query(de_duplication)
q.result();

In [ ]:
## this code appends JSON output data to BigQuery table applovinMAX_revenue_report
## inside dashboard_page9 dataset 

job = bq_client.load_table_from_json(report_data['results'],'app-swim-bi.dashboard_page9.applovinMAX_revenue_report')
job.result(); ## WAIT until job is done.

## Ironsource Data Pull

In [ ]:
##### IRON SOURCE MEDIATION PLATFORM REPORTING DATA TO BIGQUERY TABLE ##########
## url provided by Ironsource reporting API
username = '----'
secret_key = '---'
#AUID = username+':'+secret_key
response = requests.get('https://platform.ironsrc.com/partners/publisher/auth', headers={'secretkey':'---','refreshToken':'---'})
bearer_key = response.text.replace('"','')
#irn_key = base64.b64encode(AUID.encode()).decode()

## see https://developers.ironsrc.com/ironsource-mobile/air/metrics-definition/ for supported metrics
## see https://developers.ironsrc.com/ironsource-mobile/air/supported-breakdown-metric/ for supported breakdowns available

In [ ]:
headers={"Authorization": "Bearer %s" % bearer_key}
today = datetime.date.today()

startDate = today - datetime.timedelta(days=3)
startDate = datetime.date.strftime(startDate, format='%Y-%m-%d')

endDate = datetime.date.strftime(today, format='%Y-%m-%d')

irn_url = 'https://platform.ironsrc.com/partners/publisher/mediation/applications/v6/stats?startDate='+startDate+'&endDate='+endDate+\
          '&columns=revenue,eCPM,impressions,clicks,clickThroughRate'+\
          '&breakdown=country,adSource,platform,app'

resp = requests.get(irn_url, headers=headers)
irn_data = json.loads(resp.text)

In [ ]:
master = []
for each in irn_data:
  adUnit = each['adUnits']
  adNetwork = each['providerName']
  platform = each['platform']
  appKey = each['appKey']
  appName = each['appName']
  bundleId = each['bundleId']
  date = each['date']
  data = each['data']
  for dataobj in data:
    if 'videoCompletions' in dataobj:
      del dataobj['videoCompletions']
    if 'revenuePerCompletion' in dataobj:
      del dataobj['revenuePerCompletion']
    if 'completionRate' in dataobj:
      del dataobj['completionRate']
    if 'offerCompletions' in dataobj:
      del dataobj['offerCompletions']
    

    dataobj['addUnits'] = adUnit
    dataobj['adNetwork'] = adNetwork
    dataobj['platform'] = platform
    dataobj['appName'] = appName
    dataobj['bundleId'] = bundleId
    dataobj['appKey'] = appKey
    dataobj['date'] = date
    master.append(dataobj)

In [ ]:
## this code first query the table and selects only
## unique rows. Then write the results back to the table
## this way we removed duplicates in the data easily
#job_config = bigquery.QueryJobConfig(destination='app-swim-bi.dashboard_page9.ironsource_revenue_report', write_disposition='write_truncate')
de_duplication = "Delete `app-swim-bi.dashboard_page9.ironsource_revenue_report`\
                    Where date >= '"+str(startDate)+ "'  AND date <= '" +str(endDate)+"'"
q = bq_client.query(de_duplication)
q.result();

In [ ]:
## this code appends JSON output data to BigQuery table applovinMAX_revenue_report
## inside dashboard_page9 dataset 

job = bq_client.load_table_from_json(master,'app-swim-bi.dashboard_page9.ironsource_revenue_report')
job.result(); ## WAIT until job is done.

## Appfigures Data Pull

In [ ]:
client_key = '----'
encodedcreds = base64.b64encode(b'----')

In [ ]:
headers={"Authorization": "Basic %s" % encodedcreds.decode()}
today = str(datetime.date.today())
startDate = today - datetime.timedelta(days=3)
startDate = datetime.date.strftime(startDate, format='%Y-%m-%d')

endDate = datetime.date.strftime(today, format='%Y-%m-%d')
url = 'https://api.appfigures.com/v2/reports/revenue?start_date='+startDate+'&end_date='+endDate+'&group_by=date,product,store,country&format=JSON&client_key='+client_key
 
resp = requests.get(url, headers=headers)
appfig_data = json.loads(resp.text)

In [ ]:
headers={"Authorization": "Basic %s" % encodedcreds.decode()}
url = 'https://api.appfigures.com/v2/products/mine?&client_key='+client_key
resp = requests.get(url, headers=headers)
prod_info = json.loads(resp.text)

In [ ]:
master = []
for edate in appfig_data.keys():
  for eprod in appfig_data[edate].keys():
    for estore in appfig_data[edate][eprod].keys():
      for ecountry in appfig_data[edate][eprod][estore].keys():
        dataobj = copy.deepcopy(appfig_data[edate][eprod][estore][ecountry])
        dataobj['product_name'] = prod_info[eprod]['name']
        master.append(dataobj)

In [ ]:
de_duplication = "Delete `app-swim-bi.dashboard_page9.appfigures_data`\
                    Where date >= '"+str(startDate)+ "'  AND date <= '" +str(endDate)+"'"
q = bq_client.query(de_duplication)
q.result();

job = bq_client.load_table_from_json(master,'app-swim-bi.dashboard_page9.appfigures_data')
job.result(); ## WAIT until job is done.

In [ ]:
headers={"Authorization": "Basic %s" % encodedcreds.decode()}
url = 'https://api.appfigures.com/v2/reports/subscriptions?group_by=date,product,store,country&start_date='+startDate+'&end_date='+endDate+'&format=JSON&client_key='+client_key
resp = requests.get(url, headers=headers)
subs_data = json.loads(resp.text)

In [ ]:
master = []
for edate in subs_data.keys():
  for eprod in subs_data[edate].keys():
    for estore in subs_data[edate][eprod].keys():
      for ecountry in subs_data[edate][eprod][estore].keys():
        dataobj = copy.deepcopy(subs_data[edate][eprod][estore][ecountry])
        dataobj['product_name'] = prod_info[eprod]['name']
        master.append(dataobj)

In [ ]:
de_duplication = "Delete `app-swim-bi.dashboard_page9.appfigures_subscription_data`\
                    Where date >= '"+str(startDate)+ "'  AND date <= '" +str(endDate)+"'"
q = bq_client.query(de_duplication)
q.result();

job = bq_client.load_table_from_json(master,'app-swim-bi.dashboard_page9.appfigures_subscription_data')
job.result(); ## WAIT until job is done.

## Unity Ads Data Pull

In [ ]:
datetime.date.today()

datetime.date(2020, 8, 16)

In [ ]:
organization_id = "---"
access_token = "---"
url = 'https://stats.unityads.unity3d.com/organizations/---/reports/acquisitions?apikey='+access_token+\
'&start='+'2020-06-01'+'T00:00:00.000Z&end='+'2020-06-16'+'T00:00:00.000Z&scale=day&splitBy=adType,campaign,creativePack,target,store,country,platform'
  #url = 'https://stats.unityads.unity3d.com/organizations/'+organization_id+'/reports/acquisitions?\
#start='+str(startDate)+'T00:00:00.000Z&end='+str(endDate)+'T23:59:59.000Z&scale=day&\
#splitBy=adType,campaign,creativePack,target,sourceAppId,store,country,platform,osVersion,reachExtension&apikey='+access_token
resp = requests.get(url)
master = []
if 'error' not in resp.text and resp.text != '':
  csv_content = csv.reader(resp.content.decode('utf-8').splitlines(), delimiter=',')
  for each in csv_content:
    #list(map(lambda x,y: master_dict.update({x:y}), keys, vals))
    if (len(master)>1) and master[0]==each:
      pass
    else:
      master.append(each)
else:
  print("exception at Date: ", resp.text, url)  

#print(startDate, endDate)
temp = copy.deepcopy(master[0])
header_row = []
for each in temp:
  header_row.append(each.replace(' ',''))
header_row[0] = 'timestamp'
master = master[1:]
db = pd.DataFrame(master, columns=header_row)
db.insert(loc=0, column='date', value=db['timestamp'].apply(lambda x: pd.Timestamp(x).date() ))
db.drop('timestamp', axis=1, inplace=True)

In [ ]:
db['starts'] = db['starts'].astype(int)
db['views'] = db['views'].astype(int)
db['clicks'] = db['clicks'].astype(int)
db['installs'] = db['installs'].astype(int)
db['spend'] = db['spend'].astype(float)
db['cvr'] = db['cvr'].astype(float)
db['ctr'] = db['ctr'].replace('',None).astype(float)
db['ecpm'] = db['ecpm'].astype(float)
db['cpi'] = db['cpi'].replace('',0.0).astype(float)

db.head()

In [ ]:
de_duplication = "Delete `app-swim-bi.unity_dataset.unity_data`\
                    Where date >= '"+str('2020-06-01')+ "'  AND date <= '" +str('2020-06-30')+"'"
q = bq_client.query(de_duplication)
q.result();

job = bq_client.load_table_from_dataframe(db, destination='app-swim-bi.unity_dataset.unity_data')
job.result();

In [ ]:

db.loc[(db['date']>=pd.to_datetime('2020-08-10',format='%Y-%m-%d'))].groupby(['targetname','platform','campaignname','creativepackname']).agg({'starts':'sum', 'clicks':'sum','installs':'sum','spend':'sum'})

starts  ...   spend
targetname    platform campaignname                       creativepackname                   ...        
Hoop Kings    ios      Hoop King Initial Test             Creative1                     584  ...    1.00
                                                          Creative2                     584  ...    3.00
                                                          Creative3                     639  ...    2.00
Subway Idle   android  Subway AD CPI Campaign US Initial  200usd Playable               144  ...    0.00
                                                          Classic Playable             8381  ...   40.73
                                                          Earnx2 Playable             12759  ...   57.26
                                                          Sendtoapp Playable          15013  ...  100.81
                                                          Subways Creative 0 30 Sec    7784  ...   35.70
                       Subway AD CPI Campaign t3          200usd Playable             14953  ...   12.37
                                                          Classic Playable           111703  ...   97.13
                                                          Earnx2 Playable            203437  ...  165.46
                                                          Sendtoapp Playable         211650  ...  269.81
                                                          Subways Creative 0 30 Sec  100553  ...   93.23
              ios      Subway iOS CPI Campaign US Initial 200usd Playable              3066  ...   22.34
                                                          Classic Playable             1251  ...    5.08
                                                          Earnx2 Playable             30952  ...  174.81
                                                          IS1                           118  ...    0.00
                                                          IS3                            36  ...    0.53
                                                          IS4                            34  ...    0.53
                                                          Sendtoapp Playable          19250  ...  151.37
                                                          Subways Creative 0 30 Sec    3681  ...   24.99
                       Subway iOS CPI Campaign t3         200usd Playable              1669  ...    8.92
                                                          Classic Playable              947  ...    4.19
                                                          Earnx2 Playable              4018  ...   20.36
                                                          Sendtoapp Playable           4983  ...   22.87
                                                          Subways Creative 0 30 Sec    5810  ...   45.63
Water Sort 3D ios      Water Sort US Initial iOS          Creative1                    2881  ...   50.25
                                                          Creative2                    1886  ...   48.75
                                                          Creative3                    2773  ...   39.00

[29 rows x 4 columns]

In [ ]:

startDate = datetime.date(2020,8,1)
while True:
  if startDate > datetime.date(2020,8,11):
    break
  master = []
  endDate = startDate+datetime.timedelta(days=1)
  url = 'ttps://stats.unityads.unity3d.com/organizations/---/reports/acquisitions?apikey=---6&start=2020-08-13T00:00:00.000Z&end=2020-08-16T00:00:00.000Z&scale=day&splitBy=campaign,creativePack'
  #url = 'https://stats.unityads.unity3d.com/organizations/'+organization_id+'/reports/acquisitions?\
#start='+str(startDate)+'T00:00:00.000Z&end='+str(endDate)+'T23:59:59.000Z&scale=day&\
#splitBy=adType,campaign,creativePack,target,sourceAppId,store,country,platform,osVersion,reachExtension&apikey='+access_token
  resp = requests.get(url)
  if 'error' not in resp.text and resp.text != '':
    csv_content = csv.reader(resp.content.decode('utf-8').splitlines(), delimiter=',')
    for each in csv_content:
      if flag:
        list(map(lambda x,y: master_dict.update({x:y}), keys, vals))
  else:
    print("exception at Date: ", startDate, resp.text, url)  
  
  print(startDate, endDate)
  temp = copy.deepcopy(master[0])
  header_row = []
  for each in temp:
    header_row.append(each.replace(' ',''))
  header_row[0] = 'timestamp'
  master = master[1:]
  db = pd.DataFrame(master, columns=header_row)
  # db['cvr'] = db['cvr'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['ctr'] = db['ctr'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['cpi'] = db['cpi'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['ecpm'] = db['ecpm'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['installs'] = db['installs'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['starts'] = db['starts'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['views'] = db['views'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['clicks'] = db['clicks'].replace('Infinity','0.0').replace('','0.0').astype(float)
  # db['spend'] = db['spend'].replace('Infinity','0.0').replace('','0.0').astype(float)
  db.insert(loc=0, column='date', value=db['timestamp'].apply(lambda x: pd.Timestamp(x).date() ))
  db.drop('timestamp', axis=1, inplace=True)
  
  #de_duplication = "Delete `app-swim-bi.unity_dataset.unity_data`\
  #                  Where date >= '"+str(startDate)+ "'  AND date <= '" +str(endDate)+"'"
  #q = bq_client.query(de_duplication)
  #q.result();

  job = bq_client.load_table_from_dataframe(db, 'app-swim-bi.unity_dataset.unity_raw_data')
  job.result();
  print("Data Inserted to bigquery table!")
  startDate = startDate+datetime.timedelta(days=2)
  time.sleep(2)

In [ ]:
keys = ['timestamp', 'campaign id', 'platform', 'store']
vals = ['2020-08-12T00:00:01.000Z', '54546786432', 'android', 'google']

master_dict = {}
#[master_dict.update(each) for each in list(map(lambda x,y: {x:y}, keys, vals))]
)

[None, None, None, None]

In [ ]:
master_dict

{'campaign id': '54546786432',
 'platform': 'android',
 'store': 'google',
 'timestamp': '2020-08-12T00:00:01.000Z'}

In [ ]:
with open('csv_data_dump.txt','w') as f:
  f.write(str(master))

NameError: ignored

In [ ]:
db = pd.DataFrame(master, columns=header_row)
db['cvr'] = db['cvr'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['ctr'] = db['ctr'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['cpi'] = db['cpi'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['ecpm'] = db['ecpm'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['installs'] = db['installs'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['starts'] = db['starts'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['views'] = db['views'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['clicks'] = db['clicks'].replace('Infinity','0.0').replace('','0.0').astype(float)
db['spend'] = db['spend'].replace('Infinity','0.0').replace('','0.0').astype(float)

In [ ]:
db.insert(loc=0, column='date', value=db['timestamp'].apply(lambda x: pd.Timestamp(x).date() ))
db.drop('timestamp', axis=1, inplace=True)

In [ ]:
db.head()

,date,targetid,targetstoreid,targetname,creativepackid,creativepackname,adtype,campaignid,campaignname,country,platform,osversion,store,sourceappid,reachextension,starts,views,clicks,installs,spend,cvr,ctr,ecpm,cpi
0,2020-06-01,500029605,com.newleafstudios.metro,Subway Idle,5ea0c79ea403e7cc21cc32f2,200usd Playable,playable,5e9b178751427fb944444189,Subway AD CPI Campaign US Initial,AU,android,9,google,0102e2A6hosO,Unity,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2020-06-01,500029605,com.newleafstudios.metro,Subway Idle,5ea0c79ea403e7cc21cc32f2,200usd Playable,playable,5e9b178751427fb944444189,Subway AD CPI Campaign US Initial,CA,android,9,google,0102e2A6hosO,Unity,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-06-01,500029605,com.newleafstudios.metro,Subway Idle,5ea0c79ea403e7cc21cc32f2,200usd Playable,playable,5e9b178751427fb944444189,Subway AD CPI Campaign US Initial,CH,android,10,google,0102e2A6hosO,Unity,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-06-01,500029605,com.newleafstudios.metro,Subway Idle,5ea0c79ea403e7cc21cc32f2,200usd Playable,playable,5e9b178751427fb944444189,Subway AD CPI Campaign US Initial,DE,android,10,google,0102e2A6hosO,Unity,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-06-01,500029605,com.newleafstudios.metro,Subway Idle,5ea0c79ea403e7cc21cc32f2,200usd Playable,playable,5e9b178751427fb944444189,Subway AD CPI Campaign US Initial,DE,android,5.1,google,0102e2A6hosO,Unity,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
de_duplication = "Delete `app-swim-bi.unity_dataset.unity_data`\
                    Where date >= '"+str('2020-06-01')+ "'  AND date <= '" +str('2020-08-10')+"'"
q = bq_client.query(de_duplication)
q.result();

In [ ]:
job = bq_client.load_table_from_dataframe(db, 'app-swim-bi.unity_dataset.unity_data')
job.result();

In [ ]:
startDate = datetime.date(2020,1,1)
master = []
while True:
  if startDate > datetime.date(2020,7,1):
    break
  endDate = startDate+datetime.timedelta(days=6)
  irn_url = 'https://platform.ironsrc.com/partners/publisher/mediation/applications/v6/stats?startDate='+datetime.date.strftime(startDate,format='%Y-%m-%d')+'&endDate='+datetime.date.strftime(endDate,format='%Y-%m-%d')+\
          '&columns=revenue,eCPM,impressions,clicks,clickThroughRate'+\
          '&breakdown=country,adSource,platform,app'
  resp = requests.get(irn_url, headers=headers)
  irn_data = []
  if resp.text != '':
    irn_data = json.loads(resp.text)
  else:
    print(startDate, '-', endDate)
  
  if irn_data != []:
    for each in irn_data:
      adUnit = each['adUnits']
      adNetwork = each['providerName']
      platform = each['platform']
      appKey = each['appKey']
      appName = each['appName']
      bundleId = each['bundleId']
      date = each['date']
      data = each['data']
      for dataobj in data:
        if 'videoCompletions' in dataobj:
          del dataobj['videoCompletions']
        if 'revenuePerCompletion' in dataobj:
          del dataobj['revenuePerCompletion']
        if 'completionRate' in dataobj:
          del dataobj['completionRate']
        if 'offerCompletions' in dataobj:
          del dataobj['offerCompletions']
        

        dataobj['addUnits'] = adUnit
        dataobj['adNetwork'] = adNetwork
        dataobj['platform'] = platform
        dataobj['appName'] = appName
        dataobj['bundleId'] = bundleId
        dataobj['appKey'] = appKey
        dataobj['date'] = date
        master.append(dataobj)

  print(startDate, '-', endDate)
  startDate= endDate + datetime.timedelta(days=1)
  time.sleep(20)
## waste ##

2020-01-01 - 2020-01-07
2020-01-08 - 2020-01-14
2020-01-15 - 2020-01-21
2020-01-22 - 2020-01-28
2020-01-29 - 2020-02-04
2020-02-05 - 2020-02-11
2020-02-12 - 2020-02-18
2020-02-19 - 2020-02-25
2020-02-26 - 2020-03-03
2020-03-04 - 2020-03-10
2020-03-11 - 2020-03-17
2020-03-18 - 2020-03-24
2020-03-25 - 2020-03-31
2020-04-01 - 2020-04-07
2020-04-08 - 2020-04-14
2020-04-15 - 2020-04-21
2020-04-22 - 2020-04-28
2020-04-29 - 2020-05-05
2020-05-06 - 2020-05-12
2020-05-13 - 2020-05-19
2020-05-20 - 2020-05-26
2020-05-27 - 2020-06-02
2020-06-03 - 2020-06-09
2020-06-10 - 2020-06-16
2020-06-17 - 2020-06-23
2020-06-24 - 2020-06-30
2020-07-01 - 2020-07-07
